In [1]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

splits = {'train': 'sent_train.csv', 'validation': 'sent_valid.csv'}
df = pd.read_csv("hf://datasets/zeroshot/twitter-financial-news-sentiment/" + splits["train"])

In [2]:
df

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2
9539,Tupperware Brands among consumer gainers; Unil...,2
9540,vTv Therapeutics leads healthcare gainers; Myo...,2
9541,"WORK, XPO, PYX and AMKR among after hour movers",2


In [3]:
TICKER_PATTERN = re.compile(r'\$[A-Z]+')
URL_PATTERN = re.compile(r'(https:\/\/[^\s]+)')


def row_parser(row: str) -> dict:
    tickers = re.findall(TICKER_PATTERN, row)
    url = re.findall(URL_PATTERN, row)
    return {'url': '' if len(url) == 0 else url[0], 'tickers': tickers}


parsed_list = []
for row in df.text:
    parsed_list.append(row_parser(row))
parsed_df = pd.DataFrame(parsed_list)

In [4]:
df = df.join(parsed_df)
df

,text,label,url,tickers
0,$BYND - JPMorgan reels in expectations on Beyo...,0,https://t.co/bd0xbFGjkT,[$BYND]
1,$CCL $RCL - Nomura points to bookings weakness...,0,https://t.co/yGjpT2ReD3,"[$CCL, $RCL]"
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0,https://t.co/KN1g4AWFIb,[$CX]
3,$ESS: BTIG Research cuts to Neutral https://t....,0,https://t.co/MCyfTsXc2N,[$ESS]
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0,https://t.co/z37IJmCQzB,[$FNKO]
...,...,...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2,https://t.co/iK5MO4NhQq,[]
9539,Tupperware Brands among consumer gainers; Unil...,2,,[]
9540,vTv Therapeutics leads healthcare gainers; Myo...,2,,[]
9541,"WORK, XPO, PYX and AMKR among after hour movers",2,,[]


In [5]:
df.to_csv("./export/news_data_urls.csv")

In [22]:
def init_chrome_driver():
    options = Options()
    options.add_argument('--headless=new')
    driver = webdriver.Chrome(options=options)
    return driver


try:
    driver = init_chrome_driver()
except:
    print("Error Initializing driver")
    pass

body_text_list = []

In [ ]:
headers = {
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.75 Safari/537.36",
    'Accept-Language': 'en-US,en;h=0.9'}

# first 500
url_1000 = df['url'][:1000]
l = len(url_1000)
print(f'Total urls to process: {l}')
for i in range(l):
    url = url_1000[i]
    if len(url):
        # try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source)
        body_text_list.append({"article": soup.text})
        # except:
        #     print("Error")
        #     pass
    else:
        body_text_list.append({"article": ""})

    if i % 50 == 0:
        print(f'{i}/{l}')

    if i % 500 == 0:
        driver.close()
        try:
            driver = init_chrome_driver()
        except:
            print("Driver reinit failed")
driver.close()
print("Done!")